#### Import libraries and Enable GPU

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot
import MNISTtools
import torch.utils.data as data_utils
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


#### Import features and labels to numpy arrays. 
#### Normalize images between [-1,1] 
#### Convert labels to one-hot encoding 

In [2]:
xtrain, ltrain = MNISTtools.load(dataset='training', path = '/datasets/MNIST')
xtrain = np.transpose(xtrain)

def normalize_MNIST_images(x):
    x = x.astype(np.float32)
    x = (x - 127.5)/127.5
    return x

xtrain = normalize_MNIST_images(xtrain).reshape(60000,1,28,28)


def label2onehot(lbl):
    d = np.zeros((lbl.max() + 1, lbl.size))
    d[lbl[np.arange(0, lbl.size)], np.arange(0, lbl.size)] = 1
    return d
dtrain = label2onehot(ltrain)

dtrain = np.transpose(dtrain)
print(xtrain.shape)
print(dtrain.shape)

(60000, 1, 28, 28)
(60000, 10)


#### Load numpy arrays as tensors onto the GPU

In [3]:
img = torch.from_numpy(xtrain).float().to(device)
lbl = torch.from_numpy(dtrain).float().to(device)
'''Below is for mini batch gradient decent '''
train = data_utils.TensorDataset(img,lbl)
data_loader = data_utils.DataLoader(train, batch_size = 100, shuffle = True)
print(img.size())
print(lbl.size())
print(type(img))
print(type(lbl))

torch.Size([60000, 1, 28, 28])
torch.Size([60000, 10])
<class 'torch.Tensor'>
<class 'torch.Tensor'>


#### Set the hyper parameters and define cross entropy loss

In [4]:
#Hyperparameters
num_classes = 10
num_epochs = 2
batch_size = 500
learning_rate = 0.001

def cross_entropy(input, target):
    logsoftmax = nn.LogSoftmax(dim = 1)
    return torch.mean(torch.mean(-target * logsoftmax(input), dim=1))

#### Initialize the model 

In [5]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Linear(5*5*32, num_classes),nn.Softmax() )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.layer3(out)
        return out

model = ConvNet(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

#### Define miss classifications 

In [6]:
def miss_classifications(yinit, lbl):
    _, predicted = torch.max(yinit.data, 1)
    miss_classifications = (predicted != torch.max(lbl.data, 1)[1]).sum().item()
    return miss_classifications

#### Model Training 

In [7]:
for epoch in range(num_epochs):
    mc = 0
    for i, (images, labels) in enumerate(data_loader):
        outputs = model(images)
        loss = cross_entropy(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        mc += miss_classifications(outputs, labels)
        print ('Epoch: {}, Loss: {}'.format(epoch+1, loss.item()))
    print("Miss Classifications:", (mc/600))
torch.cuda.empty_cache()

/datasets/home/73/673/h6gupta/.local/lib/python2.7/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 1, Loss: 0.230449363589
Epoch: 1, Loss: 0.22880962491
Epoch: 1, Loss: 0.22741651535
Epoch: 1, Loss: 0.226537764072
Epoch: 1, Loss: 0.224671259522
Epoch: 1, Loss: 0.224214717746
Epoch: 1, Loss: 0.220984399319
Epoch: 1, Loss: 0.221887111664
Epoch: 1, Loss: 0.216451227665
Epoch: 1, Loss: 0.21542198956
Epoch: 1, Loss: 0.211080908775
Epoch: 1, Loss: 0.209079533815
Epoch: 1, Loss: 0.204377785325
Epoch: 1, Loss: 0.211288422346
Epoch: 1, Loss: 0.204543754458
Epoch: 1, Loss: 0.201304405928
Epoch: 1, Loss: 0.199905708432
Epoch: 1, Loss: 0.196807488799
Epoch: 1, Loss: 0.200199410319
Epoch: 1, Loss: 0.192857965827
Epoch: 1, Loss: 0.195433706045
Epoch: 1, Loss: 0.188609674573
Epoch: 1, Loss: 0.189100578427
Epoch: 1, Loss: 0.191869139671
Epoch: 1, Loss: 0.192596793175
Epoch: 1, Loss: 0.18585292995
Epoch: 1, Loss: 0.187641620636
Epoch: 1, Loss: 0.186475217342
Epoch: 1, Loss: 0.190725907683
Epoch: 1, Loss: 0.186528161168
Epoch: 1, Loss: 0.178427025676
Epoch: 1, Loss: 0.182724103332
Epoch: 1, Lo

Epoch: 1, Loss: 0.149942412972
Epoch: 1, Loss: 0.148868143559
Epoch: 1, Loss: 0.151394784451
Epoch: 1, Loss: 0.151983901858
Epoch: 1, Loss: 0.151150807738
Epoch: 1, Loss: 0.152568370104
Epoch: 1, Loss: 0.150736927986
Epoch: 1, Loss: 0.150890350342
Epoch: 1, Loss: 0.149502873421
Epoch: 1, Loss: 0.152725562453
Epoch: 1, Loss: 0.15150116384
Epoch: 1, Loss: 0.147762864828
Epoch: 1, Loss: 0.153059750795
Epoch: 1, Loss: 0.148652225733
Epoch: 1, Loss: 0.151340588927
Epoch: 1, Loss: 0.150521203876
Epoch: 1, Loss: 0.150105774403
Epoch: 1, Loss: 0.148226946592
Epoch: 1, Loss: 0.150673285127
Epoch: 1, Loss: 0.149715930223
Epoch: 1, Loss: 0.152070492506
Epoch: 1, Loss: 0.149731084704
Epoch: 1, Loss: 0.15068565309
Epoch: 1, Loss: 0.151670649648
Epoch: 1, Loss: 0.150038614869
Epoch: 1, Loss: 0.151899024844
Epoch: 1, Loss: 0.150302052498
Epoch: 1, Loss: 0.147774666548
Epoch: 1, Loss: 0.151674851775
Epoch: 1, Loss: 0.149896740913
Epoch: 1, Loss: 0.147852212191
Epoch: 1, Loss: 0.152563840151
Epoch: 1, 

Epoch: 1, Loss: 0.149334549904
Epoch: 1, Loss: 0.149928659201
Epoch: 1, Loss: 0.150155976415
Epoch: 1, Loss: 0.147822752595
Epoch: 1, Loss: 0.1503649652
Epoch: 1, Loss: 0.149174824357
Epoch: 1, Loss: 0.15182416141
Epoch: 1, Loss: 0.148892402649
Epoch: 1, Loss: 0.148349612951
Epoch: 1, Loss: 0.147953569889
Epoch: 1, Loss: 0.147395938635
Epoch: 1, Loss: 0.150196552277
Epoch: 1, Loss: 0.148250803351
Epoch: 1, Loss: 0.146808281541
Epoch: 1, Loss: 0.14916035533
Epoch: 1, Loss: 0.147721394897
Epoch: 1, Loss: 0.147987678647
Epoch: 1, Loss: 0.149943202734
Epoch: 1, Loss: 0.148092955351
Epoch: 1, Loss: 0.147313922644
Epoch: 1, Loss: 0.149752110243
Epoch: 1, Loss: 0.149140045047
Epoch: 1, Loss: 0.147629648447
Epoch: 1, Loss: 0.148479744792
Epoch: 1, Loss: 0.147033914924
Epoch: 1, Loss: 0.148367077112
Epoch: 1, Loss: 0.148477718234
Epoch: 1, Loss: 0.152222469449
Epoch: 1, Loss: 0.146712988615
Epoch: 1, Loss: 0.151110127568
Epoch: 1, Loss: 0.14834138751
Epoch: 1, Loss: 0.147477388382
Epoch: 1, Los

Epoch: 2, Loss: 0.147251039743
Epoch: 2, Loss: 0.147667467594
Epoch: 2, Loss: 0.149506658316
Epoch: 2, Loss: 0.147553905845
Epoch: 2, Loss: 0.148543909192
Epoch: 2, Loss: 0.147678330541
Epoch: 2, Loss: 0.147309228778
Epoch: 2, Loss: 0.146438598633
Epoch: 2, Loss: 0.149476259947
Epoch: 2, Loss: 0.147199332714
Epoch: 2, Loss: 0.14796179533
Epoch: 2, Loss: 0.148714542389
Epoch: 2, Loss: 0.148648306727
Epoch: 2, Loss: 0.148457124829
Epoch: 2, Loss: 0.148485377431
Epoch: 2, Loss: 0.14882722497
Epoch: 2, Loss: 0.149580925703
Epoch: 2, Loss: 0.148623645306
Epoch: 2, Loss: 0.147829860449
Epoch: 2, Loss: 0.149433240294
Epoch: 2, Loss: 0.14662745595
Epoch: 2, Loss: 0.146316662431
Epoch: 2, Loss: 0.149184197187
Epoch: 2, Loss: 0.14827285707
Epoch: 2, Loss: 0.148534193635
Epoch: 2, Loss: 0.148502066731
Epoch: 2, Loss: 0.148052424192
Epoch: 2, Loss: 0.148932307959
Epoch: 2, Loss: 0.146358028054
Epoch: 2, Loss: 0.148407697678
Epoch: 2, Loss: 0.149284243584
Epoch: 2, Loss: 0.146976619959
Epoch: 2, Lo

Epoch: 2, Loss: 0.147940203547
Epoch: 2, Loss: 0.146804124117
Epoch: 2, Loss: 0.147467434406
Epoch: 2, Loss: 0.14779959619
Epoch: 2, Loss: 0.146594956517
Epoch: 2, Loss: 0.147220343351
Epoch: 2, Loss: 0.14699472487
Epoch: 2, Loss: 0.14968252182
Epoch: 2, Loss: 0.146186649799
Epoch: 2, Loss: 0.147199898958
Epoch: 2, Loss: 0.148876324296
Epoch: 2, Loss: 0.147969648242
Epoch: 2, Loss: 0.146886229515
Epoch: 2, Loss: 0.146585449576
Epoch: 2, Loss: 0.146739408374
Epoch: 2, Loss: 0.147877126932
Epoch: 2, Loss: 0.151528894901
Epoch: 2, Loss: 0.147269621491
Epoch: 2, Loss: 0.147235319018
Epoch: 2, Loss: 0.146343424916
Epoch: 2, Loss: 0.148845404387
Epoch: 2, Loss: 0.147834822536
Epoch: 2, Loss: 0.146757707
Epoch: 2, Loss: 0.147501185536
Epoch: 2, Loss: 0.148073196411
Epoch: 2, Loss: 0.148338451982
Epoch: 2, Loss: 0.146261364222
Epoch: 2, Loss: 0.147659435868
Epoch: 2, Loss: 0.147968739271
Epoch: 2, Loss: 0.149134844542
Epoch: 2, Loss: 0.148556485772
Epoch: 2, Loss: 0.149486064911
Epoch: 2, Loss